In [2]:
import nltk
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import sqlite3
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [3]:
db_file = '/Users/Jasem Alzaabi/Desktop/Blue Marble/Nordstrom_full.db'
site = 'nordstrom'
product_table = 'ProductInfo'
review_table = 'ReviewsInfo'
product_column_mapping = {'ProductID': 'productId',
                          'ProductName': 'displayName',
                          'ProductIngredients': 'ingredientList',
                          '': 'ratingValue',
                          '': 'reviewCount',
                          '': 'parentCategory',
                          '': 'link',
                          'Brand': 'brandName',
                          'ProductPrice': 'listPrice',
                          '': 'rev_link',
                          'ProductTypeName': 'field'}
review_column_mapping = {'ProductID': 'productId',
                         'Comment': 'reviewText',
                         'StarRating': 'rating',
                         'submissionDate': 'submissionTime',
                         'ProductURL': 'link'}
def create_connection(db_file):
    """ 
    create a database connection to the SQLite database
    specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

def assign_cats(productId, included):
    """
    """
    if productId in included:
        return 1
    else:
        return 0
# create connection to db
cnx = create_connection(db_file)

# read the product and review databases to separate pandas dataframes
pdf = pd.read_sql_query("SELECT * FROM " + product_table, cnx)  # fault
rdf = pd.read_sql_query("SELECT * FROM " + review_table, cnx)
print('Shape of product dataframe', pdf.shape)
print('Shape of review dataframe', rdf.shape)


pdf.columns = [product_column_mapping[x] if x in product_column_mapping.keys() else x for x in pdf.columns]
rdf.columns = [review_column_mapping[x] if x in review_column_mapping.keys() else x for x in rdf.columns]

udf = rdf.drop_duplicates(subset=['productId'])
url_mapping = pd.Series(udf.link.values, index=udf.productId).to_dict()
pdf['link'] = pdf['productId'].map(url_mapping)

field_list = pdf['field'].unique()
for field in field_list:
    included = pdf[pdf.field == field]['productId'].tolist()
    pdf[field] = pdf.apply(lambda x: assign_cats(x['productId'], included), axis=1)


Shape of product dataframe (4876, 12)
Shape of review dataframe (262629, 8)


In [4]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None


In [8]:
sliced = rdf[:10]

In [9]:
# TextBlob implementation, returns (polarit, subjectivity) 
sliced['reviewedText'] = sliced['reviewText'].apply(sentiment_calc)
sliced

C:\Users\Jasem Alzaabi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ReviewID,productId,Title,reviewText,rating,userNickname,submissionTime,link,reviewedText
0,100749699,2922400,Perfectly ok.,"This is a perfectly adequate eye cream, but I think I need something that does more for my skin. A very good basic eye cream that I would use again but would try others first.",4,PiggyM,2018-03-06T15:32:55.000+00:00,https://shop.nordstrom.com/s/kiehls-since-1851-creamy-eye-treatment-with-avocado/2922400?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=none,"(0.3986666666666666, 0.41433333333333333)"
1,178769891,2922400,Worth every penny!,The most hydrating eye cream ever. I have been using this eye cream for more than 2 years now and I can tell a difference. My under eyes feel and look a lot firmer. The best eye cream for my night...,4,vd1992,2018-03-05T10:10:56.000+00:00,https://shop.nordstrom.com/s/kiehls-since-1851-creamy-eye-treatment-with-avocado/2922400?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=none,"(0.6666666666666666, 0.43333333333333335)"
2,111149155,2922400,Hydrating!!!,One of the best eye creams I've used so far! A little goes a long way with this product. I apply it underneath my eyes during my nighttime routine and wake up feeling hydrated. Upon first applicat...,5,pchawa,2018-03-04T21:32:54.000+00:00,https://shop.nordstrom.com/s/kiehls-since-1851-creamy-eye-treatment-with-avocado/2922400?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=none,"(0.16474999999999998, 0.5198333333333334)"
3,100672792,2922400,Very Hydrating,"I suffer from extremely dull and dry under eyes. Despite this product not helping to brighten the area, it has most certainly got rid of any dryness and makes applying concealer so much easier. If...",4,ama,2018-03-02T20:00:39.000+00:00,https://shop.nordstrom.com/s/kiehls-since-1851-creamy-eye-treatment-with-avocado/2922400?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=none,"(0.08154761904761905, 0.4952380952380953)"
4,111111960,2922400,Can't live without!,I never really used an eye cream until I got a sample of the Creamy Eye Treatment and now I can't go a day without. I love the smell and feel of it. Keeps your eyes looking fresh all day. \r\n\r\n...,5,VictoriaW,2018-03-02T16:48:23.000+00:00,https://shop.nordstrom.com/s/kiehls-since-1851-creamy-eye-treatment-with-avocado/2922400?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=none,"(0.2833333333333333, 0.53)"
5,184251307,5006787,nice for face,"S=First time using product so far it really working, i have noticed results in 1 week",5,momNana,2018-12-20T18:13:26.000+00:00,https://shop.nordstrom.com/s/estee-lauder-nutritious-micro-algae-pore-minimizing-shake-tonic-nordstrom-exclusive/5006787?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color...,"(0.15000000000000002, 0.6)"
6,182254965,5006787,Verified purchase,I’m in love with this entire line. Leaves you with a fresh feeling after use. I noticeable see a difference in my pores.,5,APhelps,2018-09-19T09:18:56.000+00:00,https://shop.nordstrom.com/s/estee-lauder-nutritious-micro-algae-pore-minimizing-shake-tonic-nordstrom-exclusive/5006787?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color...,"(0.26666666666666666, 0.5750000000000001)"
7,181366418,5006787,Love this !,This stuff is also great! I’m in love with the whole line! My skin looks great!,5,Yariz,2018-08-03T05:31:52.000+00:00,https://shop.nordstrom.com/s/estee-lauder-nutritious-micro-algae-pore-minimizing-shake-tonic-nordstrom-exclusive/5006787?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color...,"(0.6875, 0.625)"
8,111542056,2922400,AMAZING!!!,I’ve been using this product for a few years now and absolutely LOVE it. It’s amazing on how well it hydrates and leaves your skin feeling and looking great! It’s pricey but a little really does g...,5,Veronika,2018-03-21T05:35:35.000+00:00,https://shop.nordstrom.co

In [13]:
# VADER implementation
sliced['reviewedText'] = sliced['reviewText'].apply(sentiment_analyzer_scores)
print(sliced)

This is a perfectly adequate eye cream, but I think I need something that does more for my skin. A very good basic eye cream that I would use again but would try others first. {'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'compound': 0.8095}
The most hydrating eye cream ever. I have been using this eye cream for more than 2 years now and I can tell a difference. My under eyes feel and look a lot firmer. The best eye cream for my night time routine. {'neg': 0.0, 'neu': 0.898, 'pos': 0.102, 'compound': 0.6369}
One of the best eye creams I've used so far! A little goes a long way with this product. I apply it underneath my eyes during my nighttime routine and wake up feeling hydrated. Upon first application I noticed that it creates a wet look, however it absorbs pretty fast. I highly recommend this product and plan to repurchase it in the future! {'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'compound': 0.9243}
I suffer from extremely dull and dry under eyes. Despite this product not helping to br

C:\Users\Jasem Alzaabi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
